In [4]:

import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer  #stem word

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [5]:
lemmatizer=WordNetLemmatizer()
with open('/content/sample_data/intents.json','r') as file:
  intents=json.load(file)


In [6]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

words=[]
classes=[]
documents=[]
ignore_letters=['?','!',',','.',';']
for intent in intents['intents']:
   for pattern in intent['patterns']:
     word_list=nltk.word_tokenize(pattern)
     words.extend(word_list)
     documents.append((word_list,intent['tag']))
     if intent['tag'] not in classes:
      classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [7]:
print(documents)

[(['hello'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going'], 'greetings'), (['cya'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['Iam', 'leaving'], 'goodbye'), (['have', 'a', 'good', 'day'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['how', 'old', 'are', 'you'], 'age'), (['how', 'old', 'is', 'Hala', '?'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['age', '?'], 'age'), (['how', 'old'], 'age'), (['what', 'is', 'your', 'name', '?'], 'name'), (['what', 'should', 'i', 'call', 'you', '?'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['when', 'are', 'you', 'guys', 'open'], 'hours'), (['what', 'are', 'your', 'hours', '?'], 'hours'), (['hours', 'of', 'operation'], 'hours'), (['who', 'are', 'you', '?'], 'hou

In [8]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

In [9]:
print(words)

["'s", 'Hala', 'Iam', 'a', 'about', 'age', 'are', 'call', 'can', 'cao', 'creator', 'cya', 'day', 'drink', 'eat', 'going', 'good', 'goodbye', 'greeting', 'guy', 'have', 'hello', 'hi', 'hour', 'how', 'i', 'is', 'it', 'later', 'leaving', 'made', 'me', 'more', 'name', 'of', 'old', 'open', 'operation', 'see', 'should', 'sleep', 'something', 'tell', 'up', 'what', 'whats', 'when', 'who', 'ya', 'you', 'your']


In [10]:
classes=sorted(set(classes))


In [11]:
pickle.dump(words,open("words.pkl",'wb'))
pickle.dump(classes,open("classes.pkl",'wb'))


In [12]:
training=[]
output_empty=[0]*len(classes)
for document in documents:
  bag=[]
  word_pattern=document[0]
  word_pattern=[lemmatizer.lemmatize(word.lower()) for word in word_pattern]
  for word in words:
    bag.append(1) if word in word_pattern else bag.append(0)
  output_row=list(output_empty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])

In [13]:




random.shuffle(training)
training=np.array(training)
train_x=list(training[:,0])
train_y=list(training[:,1])


<ipython-input-13-1f8711b14fb5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [14]:
from tensorflow.keras.optimizers.legacy import SGD
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd)


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [15]:
his=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbotmodel.h5',his)


Epoch 1/200
8/8 [==============================] - 1s 3ms/step - loss: 1.9607
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9590
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9576
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 1.8077
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7768
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6529
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6524
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5013
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5018
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3164
Epoch 11/200
8/8 [==============================] - 0s 3ms/step - loss: 1.4115
Epoch 12/200
8/8 [==============================] - 0s 2ms/step - loss: 1.2325
Epoch 13/200
8/8 [==============================] - 0s 2ms/st

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from tensorflow.keras.models import load_model
lemmatizer=WordNetLemmatizer()

with open('/content/sample_data/intents.json','r') as file:
  intents=json.load(file)

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbotmodel.h5')


In [17]:
#cleaning function
def clean_up(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word)for word in sentence_words]
  return sentence_words

In [18]:
#convert sentence to bag of words
def bag_of_words(sentence):
  sentence_words=clean_up(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i ,word in enumerate(words):
      if word==w:
        bag[i]=1
  return np.array(bag)



In [19]:
def predict_class(sentence):
  x=bag_of_words(sentence)
  res=model.predict(np.array([x]))[0]
  EROR=0.25
  result=[[i,r] for i,r in enumerate(res) if r>EROR]
  result.sort(key=lambda x:x[1],reverse=True) #sort by probabilty
  return_list=[]
  for r in result:
    return_list.append({'intent':classes[r[0]],'probabilty':str(r[1])})
  return return_list

In [20]:
def get_response(intents_list,intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['responses'])
      break
  return result

In [ ]:
print("go,bot is ready")
while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

go,bot is ready
hi
1/1 [==============================] - 0s 30ms/step
Hello!
what is your name
1/1 [==============================] - 0s 20ms/step
You can call me ZI,I'm ZI,Iam ZI the assistant of Hala!
how old are you
1/1 [==============================] - 0s 21ms/step
My owner Hala is 20 years old,20 years old!
who is your creator
1/1 [==============================] - 0s 20ms/step
Hala created me!, a smart student designed me to strength her knowledge and skills in machine learning
can you eat
1/1 [==============================] - 0s 40ms/step
I don't have a physical form or biological needs like sleeping, eating, or drinking,Iam an artificial intelligence program
see you
1/1 [==============================] - 0s 22ms/step
Sad to see you go:(
